# Load tapes and lob data

do this for each day otherwise memory (RAM) exceeds most computers

In [1]:
# code
from fast_tools import get_data, get_data_gen

#data = get_data()

# Clean Data
Remove outliers from lob and create an additional columns noting this

FFill tapes data to get the most up to date tapes price

In [2]:
# code
import numpy as np
from numba import njit, prange

def get_tapes_window(tapes):
    dt = 60*60 # in seconds
    #stds = []
    #means = []
    w_bids = []
    w_asks = []

    t_start = 0
    start_time = 0
    z = 3.29 # 99.9%

    outside = []
    while True:
        end_time = start_time + dt
        t_end = t_start
        rolling_tapes = []
        while tapes[t_end, 0] < end_time:
            rolling_tapes += [tapes[t_end, 1]] * int(tapes[t_end,2])
            t_end += 1

        mean = np.mean(rolling_tapes)
        std = np.std(rolling_tapes)
        #means.append(mean)

        w_bid = mean - std * z
        w_ask = mean + std * z
        w_bids.append(w_bid)
        w_asks.append(w_ask)

        # look one minute a head
        local_end = t_end
        future_tapes = []
        while tapes[local_end, 0] < end_time + 60:
            future_tapes += [tapes[local_end, 1]] * int(tapes[local_end,2])
            local_end += 1

        future_tapes = np.array(future_tapes)

        n_above = len(np.where(future_tapes > w_ask)[0])
        n_below = len(np.where(future_tapes < w_bid)[0])
        if end_time % 60 !=0:
            raise ValueError
        outside.append((end_time,n_above, n_below, len(future_tapes)))

        start_time += 60
        while tapes[t_start, 0] < start_time:
            t_start += 1

        end_time += dt
        if end_time >= 8.5*60*60:
            break

    return outside, w_bids, w_asks

@njit(parallel=True)
def get_features(lob_data: np.array, 
                 lob_times: np.array, 
                 tapes: np.array, 
                 time_step_s: int, 
                 window_data: np.array,
                 ab_weight = 1, 
                 median = True, 
                 ):
    """
    Calculate features from LOB and Tapes data.

    Parameters:
    -----------
    lob_data : np.array
        Array containing the limit order book (LOB) data.
    lob_times : np.array
        Array containing timestamps for the LOB data.
    tapes : np.array
        Array containing Tapes data.
    time_step_s : int
        Time step in seconds for calculating features.
    ab_weight : float, optional
        Weight parameter for alpha and beta calculations, by default 1.
    median : bool, optional
        Whether to calculate features using median instead of mean, by default False.
    window_data : np.array
        Array containing window data for calculating CBS and CAS, by default None.

    Returns:
    --------
    tuple
        A tuple containing:
        - feat_arr: np.array
            Array containing feature values.
        - features: list
            List of feature names.
    """
    
    n_rows = int((8.5 * 60 * 60) / time_step_s)                         # define number of rows of output array
    features = ["MP","HIBID","LOASK","AP","WBP","WAP",                  # define features
                "TCBS","TCAS","AWS","VOL","GAP","SPREAD",
                "ALPHA", "BETA", "ZETA", "ENDT", 
                "PSTD", "LOWIN", "HIWIN", "nUoD"]
    n_features = len(features)                                          # define number of features

    feat_arr = np.zeros((n_rows, n_features), dtype=np.float64)         # array to hold feature values
    
    LA_HB_a_b = np.zeros((lob_data.shape[0]+1, 4), dtype = np.float64)  # array holding the LOASK, HIBID,
                                                                        # alpha, beta, values 

    for i in prange(lob_data.shape[0]):                                 # iterates over the LOB to fill
        row = lob_data[i]                                               # LA_HB_a_b values
        
        neg_ind = np.where(row < 0)[0]                                  # locate bid and ask prices (indicies)
        pos_ind = np.where(row > 0)[0]
        
        if len(neg_ind) == 0:                                           # assign HIBID, np.nan if no values
            LA_HB_a_b[i][1] = np.nan
        else:
            LA_HB_a_b[i][1] = max(neg_ind) + 1 

        if len(pos_ind) == 0:                                           # assign HIBID, np.nan if no values
            LA_HB_a_b[i][0] = np.nan
        else:
            LA_HB_a_b[i][0] = min(pos_ind) + 1

        mid_price = (LA_HB_a_b[i][0] + LA_HB_a_b[i][1]) / 2             # calculate mid_price for alpha/beta calculations

        if np.isnan(mid_price):
            alpha = np.nan
            beta = np.nan
        else:                                                           # calculate alpha/beta using ab_weight var
            beta = 0
            for ind in neg_ind:
                beta += (-1 * row[ind]) / ((mid_price - (ind + 1)) + ab_weight)
    
            alpha = 0
            for ind in pos_ind:
                alpha += row[ind] / (((ind + 1) - mid_price) + ab_weight)
                

        LA_HB_a_b[i][2] = alpha
        LA_HB_a_b[i][3] = beta
        
    max_lob = lob_data.shape[0] - 1                                      # define max indicies for lob
    max_tapes = tapes.shape[0] - 1                                       # define max indicies for tapes
    
    start_time = 0                                                       # define start time
    lob_start = 0                                                        # define start index for lob
    tapes_start = 0                                                      # define start index for tapes
    
    cas = np.zeros(800, dtype = np.int16)                                # define an array to hold CAS values
    cbs = np.zeros(800, dtype = np.int16)                                # define an array to hold CBS values
    for row_i in range(n_rows):
        end_time = start_time + time_step_s                              # move to next time step
        lob_end = lob_start
        tapes_end = tapes_start

        # get lob end index
        while lob_times[lob_end] < end_time and lob_end < max_lob:       # move lob indicies to end time
            lob_end += 1
        
        # get tapes end index
        while tapes[tapes_end][0] < end_time and tapes_end < max_tapes:  # move tapes indicies to end time
            tapes_end += 1

        # feature calculations
        if tapes_start == tapes_end:                                     # if there is no tapes data
            AP = np.nan                                                  # set tapes features to np.nan
            VOL = np.nan
            PSTD = np.nan
            nUoD = np.nan
        else:
            tapes_slice = tapes[tapes_start:tapes_end]                   # extract tapes slice, calculate AP, VOL
            tapes_list = []
            
            for row in tapes_slice:
                for _ in range(int(row[2])):
                    tapes_list.append(row[1])

            tapes_list = np.array(tapes_list, dtype=np.int32)
            AP = np.mean(tapes_list)
            VOL = np.sum(tapes_slice[:,2])
            PSTD = np.std(tapes_list)

            tapes_price_diff = tapes_slice[:,1][1:] - tapes_slice[:,1][:-1]
            n_ups = np.sum(tapes_price_diff > 0)
            n_downs = np.sum(tapes_price_diff < 0)
            nUoD = (n_ups + 1) / (n_downs + 1) - 1

        if lob_start == lob_end:                                         # if there is no LOB data
            MP = np.nan                                                  # set lob features to np.nan
            HIBID = np.nan
            LOASK = np.nan
            SPREAD = np.nan
            TCBS = np.nan
            TCAS = np.nan
            WBP = np.nan
            WAP = np.nan
            AWS = np.nan
            ALPHA = np.nan
            BETA = np.nan
            ZETA = np.nan  
        else:
            lob_slice = lob_data[lob_start:lob_end]                       # extract slices of data 
            LA_HB_a_b_slice = LA_HB_a_b[lob_start:lob_end]                

            # midprice_calcs, alpha, beta
            if median:                                                    # calculate price features
                HIBID = np.median(LA_HB_a_b_slice[:,1])                   # using median if set to true
                LOASK = np.median(LA_HB_a_b_slice[:,0])
                ALPHA = np.median(LA_HB_a_b_slice[:,2])
                BETA = np.median(LA_HB_a_b_slice[:,3])
            else:
                HIBID = np.nanmean(LA_HB_a_b_slice[:,1])
                LOASK = np.nanmean(LA_HB_a_b_slice[:,0])
                ALPHA = np.nanmean(LA_HB_a_b_slice[:,2])
                BETA = np.nanmean(LA_HB_a_b_slice[:,3])

            MP = (HIBID + LOASK) / 2
            SPREAD = LOASK - HIBID
            ZETA = BETA - ALPHA

            if HIBID >= LOASK:
                print("WARNING: HIBID >= LOASK")

            # consolidated calcs
            cas[:] = 0                                                      # reset cas, cbs arrays for new data
            cbs[:] = 0 

            window_index = np.where(window_data[:,0] == end_time)[0]
            if len(window_index) == 1:
                w_bid = window_data[window_index[0], 4]
                w_ask = window_data[window_index[0], 5]
                LOWIN = window_data[window_index[0], 1]
                HIWIN = window_data[window_index[0], 2]
            else:
                w_bid = MP - 100
                w_ask = MP + 100
                LOWIN = 0
                HIWIN = 0

            for ci in prange(int(np.floor(w_bid) - 1), int(np.ceil(w_ask) + 2)):
                # can optimise with LOASK AND HIBID here
                                                                        # only calculate cbs between window left of MP
                cbs_vec = lob_slice[:,ci].copy() * -1                   # and less than LOASK + 100 for efficiency
                cbs_vec[cbs_vec <= 0] = 0                               # idk if this breaks things for efficiency ?:
                cbs[ci] = np.sum(np.abs(np.diff(cbs_vec))) + cbs_vec[0]

                                                                        # only calculate cas between window right of MP
                cas_vec = lob_slice[:,ci].copy()                        # and greater than HIBID - 100 for efficiency
                cas_vec[cas_vec <= 0] = 0                               # idk if this breaks things for efficiency ?:
                cas[ci] = np.sum(np.abs(np.diff(cas_vec))) + cas_vec[0]

            TCBS = np.sum(cbs)                                              # Total CBS
            TCAS = np.sum(cas)                                              # Total CAS

            if TCBS == 0:                                                   # Calculate WBP, np.nan if no activity
                WBP = np.nan
            else:
                WBP = 0
                for ci in prange(800):
                    WBP += (ci + 1) * (cbs[ci] / TCBS)

            if TCAS == 0:                                                   # Calculate WAP, np.nan if no activity
                WAP = np.nan
            else:
                WAP = 0
                for ci in prange(800):
                    WAP += (ci + 1) * (cas[ci] / TCAS)

            AWS = WAP - WBP                                                 # Activity weighted spread calc

        # feature setting
        feat_arr[row_i][features.index("AP")] = AP                          # set the values to the feat_arr
        feat_arr[row_i][features.index("VOL")] = VOL
        feat_arr[row_i][features.index("MP")] = MP
        feat_arr[row_i][features.index("HIBID")] = HIBID
        feat_arr[row_i][features.index("LOASK")] = LOASK
        feat_arr[row_i][features.index("SPREAD")] = SPREAD
        feat_arr[row_i][features.index("TCAS")] = TCAS
        feat_arr[row_i][features.index("TCBS")] = TCBS
        feat_arr[row_i][features.index("WAP")] = WAP
        feat_arr[row_i][features.index("WBP")] = WBP
        feat_arr[row_i][features.index("AWS")] = AWS 
        feat_arr[row_i][features.index("ALPHA")] = ALPHA
        feat_arr[row_i][features.index("BETA")] = BETA
        feat_arr[row_i][features.index("ZETA")] = ZETA
        feat_arr[row_i][features.index("GAP")] = MP - AP
        feat_arr[row_i][features.index("ENDT")] = end_time
        feat_arr[row_i][features.index("PSTD")] = PSTD
        feat_arr[row_i][features.index("LOWIN")] = LOWIN
        feat_arr[row_i][features.index("HIWIN")] = HIWIN
        feat_arr[row_i][features.index("nUoD")] = nUoD


        # adjust start times
        start_time = end_time                                                # Set the next start times and 
        lob_start = lob_end                                                  # indicies to the last end times / indicies
        tapes_start = tapes_end

    return feat_arr, features

window_length_s = 5 * 60
c = 0
all_features = []
for lob, lob_times, tapes in get_data_gen():
    print(c, end = "\r")
    outside, w_bids, w_asks = get_tapes_window(tapes)
    window_data = np.zeros((len(outside), 6), dtype = float)
    for i in range(len(outside)):
        window_data[i][:4] = outside[i]
        window_data[i][4] = w_bids[i]
        window_data[i][5] = w_asks[i]

    features = get_features(lob, lob_times, tapes, window_length_s, window_data)
    all_features.append(features)
    c += 1

In [3]:
import pandas as pd

dfs = [pd.DataFrame(fa, columns=f) for fa, f in all_features]
for i, df in enumerate(dfs):
    df["WMP"] = (df["WBP"] + df["WAP"]) / 2
    df["MP_diff(y)"] = df["MP"].diff(1)
    
    df["MP_perc(y)"] = df["MP_diff(y)"] / df["MP"]
    df["DAY"] = i
df.describe()

,MP,HIBID,LOASK,AP,WBP,WAP,TCBS,TCAS,AWS,VOL,...,ZETA,ENDT,PSTD,LOWIN,HIWIN,nUoD,WMP,MP_diff(y),MP_perc(y),DAY
count,102.000000,102.000000,102.000000,102.000000,102.000000,102.000000,102.000000,102.000000,102.000000,102.000000,...,102.000000,102.000000,102.000000,102.0,102.00000,102.000000,102.000000,101.000000,101.000000,102.0
mean,119.536765,108.470588,130.602941,107.740238,97.795076,119.487203,3873.990196,1912.254902,21.692128,543.460784,...,1.049842,15450.000000,3.195585,0.0,2.22549,-0.310023,108.641140,0.084158,0.000137,124.0
std,3.015512,0.804762,5.946269,0.805628,14.389150,12.242101,2597.483106,1074.872442,26.559572,23.221564,...,0.311668,8876.654775,0.688517,0.0,6.17229,0.066626,1.451078,4.197809,0.034651,0.0
min,110.500000,107.000000,113.000000,105.154545,68.828653,110.322285,1764.000000,847.000000,5.123483,486.000000,...,0.319860,300.000000,1.885095,0.0,0.00000,-0.451327,104.793384,-14.000000,-0.119658,124.0
25%,118.000000,108.000000,127.000000,107.238569,104.255128,112.681530,2442.500000,1289.000000,7.237170,526.500000,...,0.865508,7875.000000,2.685570,0.0,0.00000,-0.352801,108.259929,-2.500000,-0.020921,124.0
50%,119.000000,108.000000,130.000000,107.759181,105.233042,113.300865,2568.000000,1407.500000,7.622364,543.500000,...,1.010785,15450.000000,3.069011,0.0,0.00000,-0.308801,109.060159,-0.500000,-0.004149,124.0
75%,121.000000,109.000000,134.000000,108.303382,105.873152,114.310386,2952.250000,1624.500000,8.823995,562.500000,...,1.215929,23025.000000,3.624244,0.0,0.00000,-0.267997,109.606310,2.500000,0.020492,124.0
max,131.000000,110.000000,154.000000,109.240530,106.726263,148.712542,9843.000000,4752.000000,76.011881,607.000000,...,2.451998,30600.000000,5.237255,0.0,38.00000,-0.125000,110.706601,12.500000,0.095420,124.0


# Extract features from LOB and Tapes

get mean, std, trend, delta from 60 min segments

In [4]:
# code

features = ["TCAS", "TCBS", "ALPHA", "BETA", "ZETA",
            "WMP", "AWS", "VOL", "GAP", "nUoD", "PSTD"]

sum_features = ["LOWIN", "HIWIN"]

y_feats = ["MP_diff(y)", "MP_perc(y)", "DAY"]

X_dfs = []



for c, df in enumerate(dfs):
    print(c, end = "\r")

    X_df = pd.DataFrame()

    for i in range(len(df) - 60):
        train_segment = df[i:i+60]
        test_segment = df[i+60:i+61]

        row = {}
        for f in features:
            # std
            row[f+"_std"] = np.std(train_segment[f])
            # mean
            if f != "WMP": # exclude mean midprice
                row[f+"_mean"] = np.mean(train_segment[f])
                row[f+"_close"] = train_segment[f].iloc[-1]
            # delta
            row[f+"_delta"] = train_segment[f].iloc[-1] - train_segment[f].iloc[0]
            # trend
            row[f+"_corr"] = np.corrcoef(np.arange(len(train_segment[f])), train_segment[f].to_numpy())[0, 1]

        for f in sum_features:
            row[f+"_sum"] = np.sum(train_segment[f])

        for yf in y_feats:
            row[yf] = test_segment[yf].iloc[0]
        row["ENDT"] = test_segment["ENDT"]

        X_df = pd.concat([X_df, pd.DataFrame(row)])

    X_dfs.append(X_df)

X_dfs[0]

,TCAS_std,TCAS_mean,TCAS_close,TCAS_delta,TCAS_corr,TCBS_std,TCBS_mean,TCBS_close,TCBS_delta,TCBS_corr,...,PSTD_mean,PSTD_close,PSTD_delta,PSTD_corr,LOWIN_sum,HIWIN_sum,MP_diff(y),MP_perc(y),DAY,ENDT
60,757.258123,3123.033333,2969.0,-1747.0,0.012129,1258.591600,5033.866667,3368.0,-4238.0,-0.558109,...,4.263905,4.225044,0.503992,-0.139297,0.0,0.0,-1.0,-0.003472,0,18300.0
61,736.928506,3110.666667,3974.0,-64.0,0.108697,1214.239439,4983.816667,4603.0,-4422.0,-0.526391,...,4.265887,3.839991,-0.226161,-0.203582,0.0,0.0,0.5,0.001733,0,18600.0
62,741.883573,3114.216667,4251.0,-584.0,0.188300,1102.772962,4897.600000,3852.0,-4044.0,-0.501171,...,4.273627,4.530524,0.082549,-0.199601,0.0,0.0,0.5,0.001730,0,18900.0
63,709.489490,3092.383333,3525.0,487.0,0.284526,1044.954517,4824.916667,3535.0,-1895.0,-0.481697,...,4.269019,4.171545,-0.366021,-0.194739,0.0,0.0,-4.5,-0.015817,0,19200.0
64,710.587041,3088.083333,2780.0,-290.0,0.269359,1049.005054,4798.883333,3868.0,-1870.0,-0.488804,...,4.265338,4.316658,-0.292784,-0.174276,0.0,0.0,0.5,0.001754,0,19500.0
65,711.714834,3083.166667,2775.0,-780.0,0.255698,1047.723762,4768.550000,3918.0,-2889.0,-0.486961,...,4.255630,4.027003,-0.850602,-0.167162,0.0,0.0,4.0,0.013841,0,19800.0
66,720.564789,3091.866667,4077.0,-306.0,0.310935,1016.134035,4724.583333,4169.0,-3071.0,-0.459967,...,4.237763,3.805549,-1.082638,-0.156023,0.0,0.0,1.5,0.005164,0,20100.0
67,701.601022,3074.650000,3350.0,-26.0,0.383802,969.367331,4666.333333,3745.0,-1328.0,-0.434680,...,4.224589,4.097747,0.049446,-0.123430,0.0,0.0,-2.5,-0.008681,0,20400.0
68,711.081968,3053.633333,2115.0,-1184.0,0.352809,982.526632,4637.466667,3341.0,-5227.0,-0.455005,...,4.232297,4.510786,0.340767,-0.116240,0.0,0.0,1.5,0.005181,0,20700.0
69,714.043193,3040.050000,2484.0,-1272.0,0.338784,845.248413,4557.233333,3754.0,-3145.0,-0.422082,...,4.238823,4.561597,0.274195,-0.098139,0.0,0.0,1.5,0.005155,0,21000.0


In [5]:
merged_dfs = pd.concat(X_dfs[:], axis = 0)
merged_dfs.to_csv(f"final_merged_{window_length_s}.csv")

# Exploratory Model

In [6]:


clean_merge_dfs = merged_dfs.dropna(how="any")

x_features = list(X_df)
x_features.remove(y_feat)

X = clean_merge_dfs[x_features]
y = 1 / (1 + np.exp(-clean_merge_dfs[y_feat]/4))

baseline_mae = np.mean(np.abs(y - 0.5))
print("Baseline MAE: ", baseline_mae)
print("Samples: ", len(merged_dfs))

NameError: name 'y_feat' is not defined

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Assuming X contains your input features and y contains your target values
# X shape: (number of samples, 35)

# Step 1: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, shuffle=True)

# Step 2: Normalize the input features
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

In [ ]:
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt

DTR = DecisionTreeRegressor(max_leaf_nodes=10)
DTR.fit(X_train, y_train)

ypred = DTR.predict(X_test)
print(mean_absolute_error(ypred, y_test))

plt.figure(figsize=(15,8))
plot_tree(DTR, feature_names=x_features)
plt.show()

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(ypred)
plt.plot(y_test.to_numpy())
plt.xlim(0,1000)

In [ ]:
plt.figure(figsize=(20,5))
plt.bar(x_features, DTR.feature_importances_)
plt.xticks(rotation=90)
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

svc = SVR()
svc.fit(X_train, y_train)

ypred = SVR.predict(X_test)
print(mean_absolute_error(ypred, y_test))

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, optimizers

def create_regression_model(input_shape):
    model = models.Sequential([
        layers.Dense(64, activation='relu', input_shape=input_shape),
        layers.Dropout(0.2),  # Add Dropout layer
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(64, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.2),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation = 'sigmoid')  # Output layer with single neuron for regression
    ])

    opt = optimizers.Adam(learning_rate=0.05)
    model.compile(optimizer=opt, loss='mean_absolute_error', metrics=['mae'])
    return model

input_shape = (len(x_features),)
regression_model = create_regression_model(input_shape)

# Display model architecture
regression_model.summary()

In [ ]:
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)
history = regression_model.fit(X_train_normalized, y_train, epochs=1000, batch_size=512, validation_split=0.5, callbacks=[early_stopping])

In [ ]:
import matplotlib.pyplot as plt
# Plot training loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.axhline(baseline_mae, c = "black", linestyle = "--")
plt.show()

test_loss = regression_model.evaluate(X_test_normalized, y_test)
model_filename = f"regression_model_test_score_{test_loss[-1]:.4f}.keras"
regression_model.save(model_filename)
# Print the test loss
print("Test Loss:", test_loss)

# Save features

for use in models

In [ ]:
merged_dfs.to_csv("final_merged_5s.csv")


In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

merged_dfs = pd.read_csv("final_merged_15s.csv", index_col=0)
plt.figure(figsize=(50,40))
sns.heatmap(merged_dfs.corr(), vmin = -1, vmax = 1)

In [ ]:
from sklearn.decomposition import PCA

merged_dfs = pd.read_csv("final_merged_15s.csv", index_col=0)
x_features = list(merged_dfs)
x_features.remove("MP_diff(y)")
x_features.remove("LOWIN_sum")

clean_dfs = merged_dfs.dropna(how="any")

for col in x_features:
    std = clean_dfs[col].std()
    mean = clean_dfs[col].mean()

    clean_dfs[col] = (clean_dfs[col] - mean) / std

pca = PCA(n_components=20)
pca_results = pca.fit_transform(clean_dfs[x_features])

plt.plot(pca.explained_variance_ratio_)

In [ ]:
import numpy as np
plt.figure(figsize=(30,30))
plt.scatter(pca_results[:,0], pca_results[:,1], c = np.log(clean_dfs["MP_diff(y)"]))